Things to do
- Preprocess images
- Load images
- Set up data for training
- Set up feature learning
- Add classification
- Receive video feed
- Process video feed



In [172]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import pandas as pd
import cv2
import tensorflow_datasets as tfds

### Setting the path to the dataset

In [173]:

import pathlib

data_dir = pathlib.Path("/kaggle/")
print(data_dir)

/kaggle


### Creating the dataset

##### About the dataset
We chose to split the test folder into 80% for testing and 20% for validation


In [174]:
#TODO: We should consider trying out different batch sizes
import tensorflow as tf
import cv2
from PIL import Image

data=[]
labels=[]

batch_size = 32
img_height = 32
img_width = 32
img_size = (img_height, img_width)

height = 32
width = 32
channels = 3
classes = 43
n_inputs = height * width*channels

for i in range(classes) :
    path = "./../kaggle/Train/{0}/".format(i)
    Class=os.listdir(path)
    for a in Class:
        try:
            image=cv2.imread(path+a)
            image_from_array = Image.fromarray(image, 'RGB')
            size_image = image_from_array.resize((height, width))
            data.append(np.array(size_image))
            labels.append(i)
        except AttributeError:
            print(" ")

Cells=np.array(data)
labels=np.array(labels)

#Randomize the order of the input images
s=np.arange(Cells.shape[0])
np.random.seed(43)
np.random.shuffle(s)
Cells=Cells[s]
labels=labels[s]

#Spliting the images into train and validation sets
(X_train,X_val)=Cells[(int)(0.2*len(labels)):],Cells[:(int)(0.2*len(labels))]
X_train = X_train.astype('float32')/255
X_val = X_val.astype('float32')/255
(y_train,y_val)=labels[(int)(0.2*len(labels)):],labels[:(int)(0.2*len(labels))]

#Using one hote encoding for the train and validation labels
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, 43)
y_val = to_categorical(y_val, 43)

# train_ds = tf.keras.utils.image_dataset_from_directory(
#     "./../kaggle/Train/",
#     validation_split=0.2,
#     subset="training",
#     seed=123,
#     image_size=img_size,
#     batch_size=batch_size
# )
#
# validation_ds = tf.keras.utils.image_dataset_from_directory(
#     "./../kaggle/Train/",
#     validation_split=0.2,
#     subset="validation",
#     seed=123,
#     image_size=img_size,
#     batch_size=batch_size
# )

# print(validation_ds)
#
# num_classes = len(train_ds.class_names)

import matplotlib.pyplot as plt

# For visualizing images in one batch
# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#     for i in range(32):
#         ax = plt.subplot(15, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(train_ds.class_names[labels[i]])
#         plt.axis("off")
#


### Standardizing the data
#### About
In this section we will standardize the images since images comes in all types shapes.



This is for rescaling the images from RGB channel [0, 255] to [0, 1]

In [175]:
#We should verify that the images actually go from [0, 255]
normalization_layer = tf.keras.layers.Rescaling(1. / 255)


### Setting up performance

In [176]:
AUTOTUNE = tf.data.AUTOTUNE

# train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
# validation_ds = validation_ds.cache().prefetch(buffer_size=AUTOTUNE)

### Training the model

In [179]:
model = tf.keras.Sequential([
    ##Do we really need the normalizastion section?
    # normalization_layer,
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(rate=.3),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(rate=.5),
    tf.keras.layers.Dense(classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    # loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

# history = model.fit(
#     train_ds,
#     validation_data=validation_ds,
#     epochs=10
# )
history = model.fit(X_train, y_train, batch_size=32, epochs=10)

Epoch 1/10
981/981 [==============================] - 16s 15ms/step - loss: 2.2833 - accuracy: 0.3389

### Visualize the training

In [180]:

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(10)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

KeyError: 'val_accuracy'

### Testing/Evaluating the model

In [194]:
#Predicting with the test data


y_test = pd.read_csv("./../kaggle/Test.csv")
labels=y_test['Path']
y_test=y_test['ClassId']



data=[]

for f in labels:
    image=cv2.imread('./../kaggle/Test/'+f.replace('Test/', ''))
    image_from_array = Image.fromarray(image, 'RGB')
    size_image = image_from_array.resize((height, width))
    data.append(np.array(size_image))


X_test=np.array(data)
X_test = X_test.astype('float32')/255
y_test = to_categorical(y_test, 43)
pred = model.predict(X_test)
res = model.evaluate(X_test, y_test)
print("test loss, test acc:", res)


[[6.3890802e-17 1.8081845e-15 8.0531873e-21 ... 2.8376785e-18
  7.6002285e-13 2.8389135e-10]
 [8.0803595e-03 9.8841977e-01 1.4045749e-07 ... 7.4639919e-11
  2.8818569e-17 2.9843986e-21]
 [0.0000000e+00 0.0000000e+00 4.1607993e-33 ... 1.9606420e-21
  1.0855844e-31 1.9491284e-28]
 ...
 [6.2006810e-03 4.1803336e-03 4.9792591e-02 ... 5.9850086e-03
  7.9671224e-04 1.4204028e-03]
 [3.7756711e-06 4.4404989e-04 3.9124381e-04 ... 1.7321284e-04
  2.7762750e-09 2.6863904e-06]
 [3.2894409e-11 3.3754528e-08 1.7061826e-06 ... 2.0851422e-10
  8.6737659e-11 2.2703368e-09]]
395/395 [==============================] - 1s 4ms/step - loss: 0.2260 - accuracy: 0.9477